<a href="https://colab.research.google.com/github/emanuelaoliveira93/mfc/blob/main/Exerciciomfc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"provenance":[],"authorship_tag":"ABX9TyPnsLZNhfJm8Gvn0sLeOZzd"},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"}},"cells":[{"cell_type":"code","source":["import numpy as np\n","import matplotlib.pyplot as plt\n","import yfinance as yf\n","from scipy.optimize import minimize"],"metadata":{"id":"AgWTvzi0mCsG","executionInfo":{"status":"ok","timestamp":1730849737739,"user_tz":180,"elapsed":283,"user":{"displayName":"Pedro de Barba","userId":"12223928638106934906"}}},"execution_count":2,"outputs":[]},{"cell_type":"code","source":["# Define funções utilizadas\n","\n","def calcula_retorno(carteira, retornos):\n","    \"\"\"\n","    Calcula o retorno esperado de uma carteira dada uma série de retornos de ativos.\n","    \"\"\"\n","    return np.sum(carteira * retornos)\n","\n","def calcula_risco(carteira, covariancia):\n","    \"\"\"\n","    Calcula o risco de uma carteira dada a matriz de covariância dos ativos.\n","    \"\"\"\n","    return np.sqrt(np.dot(carteira.T, np.dot(covariancia, carteira)))\n","\n","def calcula_fronteira_eficiente(retornos, covariancia, num_pontos=500):\n","    \"\"\"\n","    Calcula a fronteira eficiente de uma carteira.\n","    \"\"\"\n","    num_ativos = len(retornos)\n","    pesos_simulados = np.random.random((num_pontos, num_ativos))\n","    pesos_simulados /= np.sum(pesos_simulados, axis=1)[:, np.newaxis]\n","\n","    riscos = np.zeros(num_pontos)\n","    retornos_carteira = np.zeros(num_pontos)\n","\n","    for i in range(num_pontos):\n","        carteira = pesos_simulados[i]\n","        retornos_carteira[i] = calcula_retorno(carteira, retornos)\n","        riscos[i] = calcula_risco(carteira, covariancia)\n","\n","    return riscos, retornos_carteira\n","\n","def calcula_minima_variancia(retornos, covariancia):\n","    \"\"\"\n","    Calcula a carteira de mínima variância dado os retornos e a matriz de covariância.\n","    \"\"\"\n","    num_ativos = len(retornos)\n","    ones = np.ones(num_ativos)\n","    cov_inv = np.linalg.inv(covariancia)\n","    w = np.dot(cov_inv, ones) / np.dot(ones, np.dot(cov_inv, ones))\n","    return w\n","\n","def calcula_carteira_otima(retornos, covariancia, ativo_livre_risco):\n","    \"\"\"\n","    Calcula a carteira ótima utilizando a otimização de Markowitz.\n","    \"\"\"\n","    num_ativos = len(retornos)\n","    w0 = np.ones(num_ativos) / num_ativos\n","    bounds = tuple((0, 1) for asset in range(num_ativos))\n","\n","    def neg_sharpe_ratio(weights, retornos, covariancia):\n","        r = calcula_retorno(weights, retornos)\n","        vol = calcula_risco(weights, covariancia)\n","        return -(r-ativo_livre_risco) / vol\n","\n","    result = minimize(neg_sharpe_ratio, w0, args=(retornos, covariancia),\n","                      method='SLSQP', bounds=bounds, constraints=({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}))\n","    return result.x"],"metadata":{"id":"RsFwUe7bjDLM","executionInfo":{"status":"ok","timestamp":1730849837435,"user_tz":180,"elapsed":248,"user":{"displayName":"Pedro de Barba","userId":"12223928638106934906"}}},"execution_count":3,"outputs":[]},{"cell_type":"code","source":["# Lista de tickers de ações\n","# tickers = ['PETR4.SA', 'BBAS3.SA']\n","tickers = ['PETR4.SA', 'VALE3.SA', 'BBAS3.SA']\n","\n","# Obtendo dados de preços de fechamento utilizando a biblioteca yfinance\n","precos_fechamento = yf.download(tickers, start='2024-01-01', end='2024-11-05')['Adj Close']\n","\n","# Calculando os retornos diários\n","retornos = precos_fechamento.pct_change().mean() * 252  # Retorna os retornos médios anualizados\n","covariancia = precos_fechamento.pct_change().cov() * 252  # Retorna a covariância anualizada\n","\n","# Plotagem do gráfico de evolução temporal de cada ticker\n","plt.figure(figsize=(10, 6))\n","for ticker in tickers:\n","    plt.plot(precos_fechamento.index, precos_fechamento[ticker], label=ticker)\n","\n","plt.title('Evolução Temporal de Preços de Fechamento')\n","plt.xlabel('Data')\n","plt.ylabel('Preço de Fechamento')\n","plt.legend()\n","plt.grid(True)\n","plt.show()\n","\n","# Imprimir o risco e o retorno de cada ativo no período\n","for ticker in tickers:\n","    retorno = precos_fechamento[ticker].pct_change().mean() * 252\n","    risco = precos_fechamento[ticker].pct_change().std() * np.sqrt(252)\n","    print(f\"Ticker: {ticker}, Retorno: {retorno:.4f}, Risco: {risco:.4f}\")\n"],"metadata":{"colab":{"base_uri":"https://localhost:8080/","height":633},"id":"oQm9oAAlm8H8","executionInfo":{"status":"ok","timestamp":1730850306025,"user_tz":180,"elapsed":1163,"user":{"displayName":"Pedro de Barba","userId":"12223928638106934906"}},"outputId":"ebe75d5b-8d44-4407-a6d5-b09a251221de"},"execution_count":6,"outputs":[{"output_type":"stream","name":"stderr","text":["[*********************100%***********************]  3 of 3 completed\n"]},{"output_type":"display_data","data":{"text/plain":["<Figure size 1000x600 with 1 Axes>"],"image/png":"...\n"},"metadata":{}},{"output_type":"stream","name":"stdout","text":["Ticker: PETR4.SA, Retorno: 0.1091, Risco: 0.2569\n","Ticker: VALE3.SA, Retorno: -0.1258, Risco: 0.2187\n","Ticker: BBAS3.SA, Retorno: 0.0271, Risco: 0.1613\n"]}]},{"cell_type":"code","source":["# Define ativo livre de risco\n","ativo_livre_risco = 0.01\n","\n","# Calcula a fronteira eficiente\n","riscos_fronteira, retornos_fronteira = calcula_fronteira_eficiente(retornos, covariancia)\n","\n","# Calcula a carteira de mínima variância\n","pesos_min_var = calcula_minima_variancia(retornos, covariancia)\n","retorno_min_var = calcula_retorno(pesos_min_var, retornos)\n","risco_min_var = calcula_risco(pesos_min_var, covariancia)\n","\n","# Calcula a carteira ótima\n","pesos_otima = calcula_carteira_otima(retornos, covariancia, ativo_livre_risco)\n","retorno_otima = calcula_retorno(pesos_otima, retornos)\n","risco_otima = calcula_risco(pesos_otima, covariancia)\n","\n","# Plota o gráfico risco versus retorno\n","plt.figure(figsize=(10, 6))\n","plt.scatter(riscos_fronteira, retornos_fronteira, c=(retornos_fronteira - ativo_livre_risco)/ riscos_fronteira, marker='o')\n","plt.scatter(risco_min_var, retorno_min_var, color='red', marker='*', s=100, label='Carteira de Mínima Variância')\n","plt.scatter(risco_otima, retorno_otima, color='green', marker='*', s=100, label='Carteira Ótima')\n","plt.title('Fronteira Eficiente')\n","plt.xlabel('Risco')\n","plt.ylabel('Retorno')\n","plt.colorbar(label='Índice de Sharpe')\n","plt.legend()\n","plt.grid(True)\n","plt.show()\n","\n","# Imprime a composição, risco e retorno da carteira de mínima variância\n","print(\"Carteira de Mínima Variância:\")\n","for ticker, peso in zip(tickers, pesos_min_var):\n","    print(f\"{ticker}: {peso:.2f}\")\n","print(f\"Retorno: {retorno_min_var:.4f}\")\n","print(f\"Risco: {risco_min_var:.4f}\")\n","\n","# Imprime a composição, risco e retorno da carteira ótima\n","print(\"\\nCarteira Ótima:\")\n","for ticker, peso in zip(tickers, pesos_otima):\n","    print(f\"{ticker}: {peso:.2f}\")\n","print(f\"Retorno: {retorno_otima:.4f}\")\n","print(f\"Risco: {risco_otima:.4f}\")\n"],"metadata":{"colab":{"base_uri":"https://localhost:8080/","height":790},"id":"8d-Wz83TnDa0","executionInfo":{"status":"ok","timestamp":1730850459035,"user_tz":180,"elapsed":1282,"user":{"displayName":"Pedro de Barba","userId":"12223928638106934906"}},"outputId":"76ce803d-2e68-42a3-a545-52222f296a7d"},"execution_count":8,"outputs":[{"output_type":"display_data","data":{"text/plain":["<Figure size 1000x600 with 2 Axes>"],"image/png":"...\n"},"metadata":{}},{"output_type":"stream","name":"stdout","text":["Carteira de Mínima Variância:\n","PETR4.SA: 0.57\n","VALE3.SA: 0.15\n","BBAS3.SA: 0.28\n","Retorno: -0.0035\n","Risco: 0.1298\n","\n","Carteira Ótima:\n","PETR4.SA: 0.14\n","VALE3.SA: 0.86\n","BBAS3.SA: 0.00\n","Retorno: 0.0977\n","Risco: 0.2261\n"]}]}]}

{'nbformat': 4,
 'nbformat_minor': 0,
 'metadata': {'colab': {'provenance': [],
   'authorship_tag': 'ABX9TyPnsLZNhfJm8Gvn0sLeOZzd'},
  'kernelspec': {'name': 'python3', 'display_name': 'Python 3'},
  'language_info': {'name': 'python'}},
 'cells': [{'cell_type': 'code',
   'source': ['import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import yfinance as yf\n',
    'from scipy.optimize import minimize'],
   'metadata': {'id': 'AgWTvzi0mCsG',
    'executionInfo': {'status': 'ok',
     'timestamp': 1730849737739,
     'user_tz': 180,
     'elapsed': 283,
     'user': {'displayName': 'Pedro de Barba',
      'userId': '12223928638106934906'}}},
   'execution_count': 2,
   'outputs': []},
  {'cell_type': 'code',
   'source': ['# Define funções utilizadas\n',
    '\n',
    'def calcula_retorno(carteira, retornos):\n',
    '    """\n',
    '    Calcula o retorno esperado de uma carteira dada uma série de retornos de ativos.\n',
    '    """\n',
    '    return np.sum(carteir